### 1. Clustering
#### 1-1

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc =SparkContext()

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf, lit
from pyspark.sql.types import IntegerType
from pyspark import SQLContext
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
spark = SparkSession\
    .builder\
    .appName("Clustering")\
    .getOrCreate()
# dataframe1 = spark.read.json('hdfs://localhost:1234/user/tl2861/hw3/train.json')

sqlContext = SQLContext(sc)
# dataframe1 = sqlContext.read.json('hdfs://localhost:1234/user/tl2861/hw3/train.json')
dataframe1 = sqlContext.read.json('../../data/AA/wiki_*').withColumn('label',lit(1))
dataframe2 = sqlContext.read.json('../../data/AAquote/wiki_*').withColumn('label', lit(2))
dataframe3 = sqlContext.read.json('../../data/AAvoyage/wiki_*').withColumn('label',lit(3))
dataframe = dataframe1.unionAll(dataframe2).unionAll(dataframe3)
# dataframe1.printSchema()
# dataframe2.printSchema()
dataframe.printSchema()
dataframe1.show()
#regexTokenizer = RegexTokenizer()
sampledData = dataframe.sampleBy("label", fractions={1: 0.2, 2: 0.2, 3:0.2}, seed=0)
sampledData.show()

root
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- label: integer (nullable = false)

+------+--------------------+--------------------+--------------------+-----+
|    id|                text|               title|                 url|label|
+------+--------------------+--------------------+--------------------+-----+
|204805|BP report into Gu...|BP report into Gu...|https://en.wikine...|    1|
|204818|200 candles: Chil...|200 candles: Chil...|https://en.wikine...|    1|
|204838|New flotilla plan...|New flotilla plan...|https://en.wikine...|    1|
|205039|Large gas main ex...|Large gas main ex...|https://en.wikine...|    1|
|205165|Air Zimbabwe pilo...|Air Zimbabwe pilo...|https://en.wikine...|    1|
|205171|Police may have k...|Police may have k...|https://en.wikine...|    1|
|205326|Test article for ...|Test article for ...|https://en.wikine...|    1|
|205328|Another test arti...|Anot

In [3]:
# sampledData.createOrReplaceTempView("wikinews")
# # sampledData = spark.sql("select count(*) from wikinews")
# sampledData.show()
sampledData.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 4381|
|    2| 6586|
|    3| 5687|
+-----+-----+



In [4]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="[^A-Za-z]+", toLowercase=True)
tokenizedData = regexTokenizer.transform(sampledData)

In [5]:
stopWordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filteredData = stopWordsRemover.transform(tokenizedData)

In [6]:
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=20)
featurizedData = hashingTF.transform(filteredData)

In [7]:
idf= IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(featurizedData)

In [8]:
data1 = idfModel.transform(featurizedData)
datatext = data1.select('features')
datatext.show()

+--------------------+
|            features|
+--------------------+
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[13],[0.29251...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,2,3,4,5,6,...|
|(20,[0,1,2,3,4,5,...|
|(20,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



#### k-means

In [9]:
from pyspark.ml.clustering import KMeans

In [14]:
kmeans = KMeans().setK(3).setSeed(1)
kmModel = kmeans.fit(data1)
wssse = kmModel.computeCost(data1)
print("Within Set Sum of Squared Errors = " + str(wssse))
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
     print(center)

Within Set Sum of Squared Errors = 5588951.555903378
Cluster Centers: 
[1.90218591 1.76718211 1.9384885  1.75734003 1.85523991 1.60344867
 1.75974671 1.78994292 2.09537133 1.71989525 1.84462279 1.71652649
 1.65677558 2.04217616 1.78857564 1.92950761 1.84122447 1.84168953
 1.65142181 1.76094185]
[216.93424333 226.97499146 236.95311999 224.68000297 247.23592687
 186.44055826 226.9477241  247.24803096 260.64449734 215.07688574
 230.91845654 224.74092657 214.78255033 276.18491303 226.63861283
 209.70512765 240.29353518 262.07779673 249.83620712 230.74621583]
[39.60137921 39.19407256 43.96566935 37.51279381 40.95349113 34.07051731
 37.33200405 40.24226937 47.05695707 35.00688492 40.35540531 37.3945405
 34.25163389 44.75369227 37.92117138 39.84130398 38.84469415 42.29642833
 36.00683019 41.33687224]


In [ ]:
sorted(kmModel.groupBy(kmModel.prediction).count().collect())

In [15]:
transformed = kmModel.transform(data1).select("label","prediction")
sorted(df.groupBy(df.age).count().collect())
# transformed.corr("label","prediction")

+-----+----------+
|label|prediction|
+-----+----------+
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
|    1|         0|
+-----+----------+
only showing top 20 rows



#### Gaussian Mixture Model (GMM)

In [11]:
from pyspark.ml.clustering import GaussianMixture

# loads data
gmm = GaussianMixture().setK(3).setSeed(538009335)
gmmModel = gmm.fit(data1)

# print("Gaussians shown as a DataFrame: ")
# model.gaussiansDF.show(truncate=False)

summary = gmmModel.summary
print (summary.k)
print (summary.logLikelihood)

3
-473262.5193272425


#### Latent Dirichlet allocation (LDA)

In [12]:
from pyspark.ml.clustering import LDA

# Trains a LDA model.
lda = LDA(k=3, maxIter=10)
ldaModel = lda.fit(data1)

ll = ldaModel.logLikelihood(data1)
lp = ldaModel.logPerplexity(data1)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics.
topics = ldaModel.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# Shows the result
# transformed = ldaModel.transform(data1)
# transformed.show(truncate=False)

The lower bound on the log likelihood of the entire corpus: -2390009.871012725
The upper bound on perplexity: 3.0418218773216754
The topics described by their top-weighted terms:
+-----+-----------+----------------------------------------------------------------+
|topic|termIndices|termWeights                                                     |
+-----+-----------+----------------------------------------------------------------+
|0    |[8, 13, 9] |[0.07043593730288787, 0.06712518106433367, 0.06675711484113477] |
|1    |[3, 6, 4]  |[0.08853821689532505, 0.08585223821989096, 0.07149464974725137] |
|2    |[2, 8, 10] |[0.062113445695059805, 0.0587201121505063, 0.055194423813886076]|
+-----+-----------+----------------------------------------------------------------+

